In [1]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 490, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 490 (delta 149), reused 124 (delta 91), pack-reused 269
Receiving objects: 100% (490/490), 136.70 KiB | 769.00 KiB/s, done.
Resolving deltas: 100% (251/251), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.4 MB/s eta 0:00:00
Installing the rest of the RAPIDS 24.4.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 GB 540.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.8/137.8 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.9/920.9 kB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import dask_cudf
import cudf
import dask
import cupy
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
from cuml.model_selection import train_test_split, GridSearchCV
from cuml.preprocessing import StandardScaler
from cuml.metrics import mean_squared_error, accuracy_score
from cuml.ensemble import RandomForestRegressor as cuRFRegressor
from cuml.linear_model import LogisticRegression
from cuml.preprocessing import LabelEncoder

In [3]:
cluster = LocalCUDACluster()


INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:41963
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45571'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:42379', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:42379
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:55070


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Read data into a Dask-cuDF DataFrame
df = dask_cudf.read_parquet('/content/drive/MyDrive/Colab Notebooks/yellow_tripdata_2024-03.parquet')
df.compute()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2024-03-01 00:18:51,2024-03-01 00:23:45,0,1.30,1,N,142,239,1,8.60,3.5,0.5,2.70,0.0,1.0,16.30,2.5,0.0
1,1,2024-03-01 00:26:00,2024-03-01 00:29:06,0,1.10,1,N,238,24,1,7.20,3.5,0.5,3.00,0.0,1.0,15.20,2.5,0.0
2,2,2024-03-01 00:09:22,2024-03-01 00:15:24,1,0.86,1,N,263,75,2,7.90,1.0,0.5,0.00,0.0,1.0,10.40,0.0,0.0
3,2,2024-03-01 00:33:45,2024-03-01 00:39:34,1,0.82,1,N,164,162,1,7.90,1.0,0.5,1.29,0.0,1.0,14.19,2.5,0.0
4,1,2024-03-01 00:05:43,2024-03-01 00:26:22,0,4.90,1,N,263,7,2,25.40,3.5,0.5,0.00,0.0,1.0,30.40,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3582623,2,2024-03-31 23:16:45,2024-03-31 23:29:20,<NA>,4.27,<NA>,<NA>,130,218,0,14.27,0.0,0.5,0.00,0.0,1.0,15.77,<NA>,<NA>
3582624,1,2024-03-31 23:29:28,2024-03-31 23:43:47,<NA>,0.00,<NA>,<NA>,79,100,0,14.00,1.0,0.5,3.80,0.0,1.0,24.80,<NA>,<NA>
3582625,2,2024-03-31 23:15:00,2024-03-31 23:47:29,<NA>,6.44,<NA>,<NA>,63,181,0,30.00,0.0,0.5,0.00,0.0,1.0,31.50,<NA>,<NA>
3582626,2,2024-03-31 23:27:53,2024-03-31 23:45:44,<NA>,3.88,<NA>,<NA>,161,148,0,22.32,0.0,0.5,5.26,0.0,1.0,31.58,<NA>,<NA>


In [7]:
numerical_var = df[['fare_amount','passenger_count', 'extra', 'mta_tax','tip_amount', 'tolls_amount', 'improvement_surcharge','total_amount', 'congestion_surcharge', 'Airport_fee']]
numerical_var.describe()

,fare_amount,passenger_count,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
npartitions=1,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...


In [8]:
numerical_var = df[['fare_amount','passenger_count', 'extra', 'mta_tax','tip_amount', 'tolls_amount', 'improvement_surcharge','total_amount', 'congestion_surcharge', 'Airport_fee']]
numerical_var_above_zero = numerical_var[numerical_var['fare_amount']>0]
numerical_var_above_zero.compute()

,fare_amount,passenger_count,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,8.60,0,3.5,0.5,2.70,0.0,1.0,16.30,2.5,0.0
1,7.20,0,3.5,0.5,3.00,0.0,1.0,15.20,2.5,0.0
2,7.90,1,1.0,0.5,0.00,0.0,1.0,10.40,0.0,0.0
3,7.90,1,1.0,0.5,1.29,0.0,1.0,14.19,2.5,0.0
4,25.40,0,3.5,0.5,0.00,0.0,1.0,30.40,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...
3582623,14.27,<NA>,0.0,0.5,0.00,0.0,1.0,15.77,<NA>,<NA>
3582624,14.00,<NA>,1.0,0.5,3.80,0.0,1.0,24.80,<NA>,<NA>
3582625,30.00,<NA>,0.0,0.5,0.00,0.0,1.0,31.50,<NA>,<NA>
3582626,22.32,<NA>,0.0,0.5,5.26,0.0,1.0,31.58,<NA>,<NA>


In [9]:
numerical_var_above_zero_no_na = numerical_var_above_zero.dropna()
numerical_var_above_zero_no_na = numerical_var_above_zero_no_na.compute()

In [11]:
correlation_matrix = numerical_var_above_zero_no_na.corr()

In [12]:
fare_amount_correlation = correlation_matrix["fare_amount"]
fare_amount_correlation

fare_amount              1.000000
passenger_count          0.045020
extra                    0.148969
mta_tax                 -0.289924
tip_amount               0.601516
tolls_amount             0.631061
improvement_surcharge   -0.013567
total_amount             0.978127
congestion_surcharge    -0.336617
Airport_fee              0.596249
Name: fare_amount, dtype: float64

In [13]:
# Select features based on correlation thresholds
threshold = 0.5  # Set your correlation threshold here
highly_correlated_features = fare_amount_correlation[fare_amount_correlation>0.5]
highly_correlated_features

fare_amount     1.000000
tip_amount      0.601516
tolls_amount    0.631061
total_amount    0.978127
Airport_fee     0.596249
Name: fare_amount, dtype: float64

In [14]:
highly_correlated_features.index.values_host

array(['fare_amount', 'tip_amount', 'tolls_amount', 'total_amount',
       'Airport_fee'], dtype=object)

In [15]:
selected_features = numerical_var_above_zero_no_na[highly_correlated_features.index.values_host]
selected_features.describe()

,fare_amount,tip_amount,tolls_amount,total_amount,Airport_fee
count,3.111328e+06,3.111328e+06,3.111328e+06,3.111328e+06,3.111328e+06
mean,1.924837e+01,3.587126e+00,6.074030e-01,2.840509e+01,1.482540e-01
std,1.807294e+01,4.107060e+00,2.238670e+00,2.282534e+01,4.873040e-01
min,1.000000e-02,0.000000e+00,0.000000e+00,1.010000e+00,0.000000e+00
25%,9.300000e+00,1.000000e+00,0.000000e+00,1.570000e+01,0.000000e+00
50%,1.280000e+01,2.860000e+00,0.000000e+00,2.060000e+01,0.000000e+00
75%,2.120000e+01,4.400000e+00,0.000000e+00,2.988000e+01,0.000000e+00
max,9.000000e+02,9.999900e+02,1.630000e+02,1.021990e+03,1.750000e+00


In [16]:
gpu_data = cupy.asarray(selected_features['fare_amount'])
gpu_data

array([ 8.6,  7.2,  7.9, ..., 12.1,  7.9,  7.9])

In [17]:
bins = [-float("inf"), 10, 20, float("inf")]
labels = ['Low', 'Medium', 'High']
selected_features['fare_discretized'] = cudf.cut(gpu_data, bins=bins, labels=labels, right=False)

In [18]:
X_reg = selected_features[['tip_amount','tolls_amount','total_amount']]
y_reg = selected_features['fare_amount']
X_cls = selected_features[['tip_amount','tolls_amount','total_amount']]
y_cls = selected_features['fare_discretized']

In [19]:
# Step 3: Training and Validation
# Regression (cuRFRegressor)
xgb_reg = cuRFRegressor()
params_reg = {'n_estimators': [5, 10], 'max_depth': [5, 7]}
grid_reg = GridSearchCV(xgb_reg, params_reg, cv=5)
grid_reg.fit(X_reg, y_reg)
best_reg_model = grid_reg.best_estimator_

# Classification (Logistic Regression)
scaler = StandardScaler()
X_cls_scaled = scaler.fit_transform(X_cls)
log_reg = LogisticRegression()
params_cls = {'C': [0.1, 1, 10]}
grid_cls = GridSearchCV(log_reg, params_cls, cv=5)
grid_cls.fit(X_cls_scaled, y_cls)
best_cls_model = grid_cls.best_estimator_

# Step 4: Testing
# Example: Split data into train and test sets
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(X_cls_scaled, y_cls, test_size=0.2, random_state=42)


selected_features_1000 = selected_features[:100000]

X_reg = selected_features[['tip_amount','tolls_amount','total_amount','Airport_fee']]
y_reg = selected_features['fare_amount']
X_cls = selected_features[['tip_amount','tolls_amount','total_amount','Airport_fee']]
y_cls = selected_features['fare_discretized']

# Evaluate regression model
reg_predictions = best_reg_model.predict(X_test_reg)
reg_rmse = mean_squared_error(y_test_reg, reg_predictions, squared=False)

# Evaluate classification model
cls_predictions = best_cls_model.predict(X_test_cls)
cls_accuracy = accuracy_score(y_test_cls, cls_predictions)
cls_report = classification_report(y_test_cls, cls_predictions)

/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


TypeError: Implicit conversion to a host NumPy array via __array__ is not allowed, To explicitly construct a GPU matrix, consider using .to_cupy()
To explicitly construct a host matrix, consider using .to_numpy().

In [20]:
import numpy as np

# Profile Code

In [24]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [25]:
!pip install yappi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.0 MB/s eta 0:00:00


## Preprocessing

In [30]:
import yappi

In [31]:
%%time
yappi.clear_stats()
# Start profiling
yappi.start()
# Step 2: Preprocessing
df = dask_cudf.read_parquet('/content/drive/MyDrive/Colab Notebooks/yellow_tripdata_2024-03.parquet')

numerical_var = df[['fare_amount','passenger_count', 'extra', 'mta_tax','tip_amount', 'tolls_amount', 'improvement_surcharge','total_amount', 'congestion_surcharge', 'Airport_fee']]
numerical_var_above_zero = numerical_var[numerical_var['fare_amount']>0]

numerical_var_above_zero_no_na = numerical_var_above_zero.dropna()
numerical_var_above_zero_no_na = numerical_var_above_zero_no_na.compute()

correlation_matrix = numerical_var_above_zero_no_na.corr()

fare_amount_correlation = correlation_matrix["fare_amount"]

threshold = 0.5  # Set your correlation threshold here
highly_correlated_features = fare_amount_correlation[fare_amount_correlation>0.5]

selected_features = numerical_var_above_zero_no_na[highly_correlated_features.index.values_host]

selected_features = selected_features.sample(n=100000, random_state=42)

gpu_data = cupy.asarray(selected_features['fare_amount'])

bins = [-float("inf"), 10, 20, float("inf")]
labels = ['Low', 'Medium', 'High']
selected_features['fare_discretized'] = cudf.cut(gpu_data, bins=bins, labels=labels, right=False)
label_encoder = LabelEncoder()

# Fit and transform the target column
selected_features['fare_discretized'] = label_encoder.fit_transform(selected_features['fare_discretized'])


X_reg = selected_features[['tip_amount','tolls_amount','total_amount']]
y_reg = selected_features['fare_amount']
X_cls = selected_features[['tip_amount','tolls_amount','total_amount']]
y_cls = selected_features['fare_discretized']

# Stop profiling
yappi.stop()

# Get and sort function stats by total time (descending)
func_stats_preproc = yappi.get_func_stats().sort('ttot', 'desc')
# Get and sort thread stats by total time (descending)
thread_stats_preproc = yappi.get_thread_stats().sort('ttot', 'desc')

CPU times: user 1.06 s, sys: 142 ms, total: 1.21 s
Wall time: 1.25 s


## XGBRegressor

In [45]:
%%time
%load_ext cudf.pandas
# Clear previous profiling stats
yappi.clear_stats()
yappi.start()
from xgboost import XGBRegressor
import xgboost as xgb

X_cls = selected_features[['tip_amount','tolls_amount','total_amount']].to_cupy()
y_cls = selected_features['fare_discretized'].to_cupy()
y_cls = y_cls.get()
#X_cls_scaled = X_cls_scaled.get()

# Convert to DMatrix (xgboost's data structure)
dtrain = xgb.DMatrix(X_cls_scaled, label=y_cls)

# Define the parameter grid for GridSearchCV
param_grid = {
    'max_depth': [5,10],
    'n_estimators': [5, 7]
}

# Initialize XGBRegressor
xgb_regressor = XGBRegressor()

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=xgb_regressor, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_cls_scaled, y_cls)

# Best parameters and model
print("Best parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_
# Stop profiling
yappi.stop()

# Get and sort function stats by total time (descending)
func_stats_xgbreg = yappi.get_func_stats().sort('ttot', 'desc')
# Get and sort thread stats by total time (descending)
thread_stats_xgbreg = yappi.get_thread_stats().sort('ttot', 'desc')

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas
Best parameters: {'max_depth': 10, 'n_estimators': 7}
CPU times: user 1.7 s, sys: 109 ms, total: 1.81 s
Wall time: 978 ms


## cuRFRegressor

In [33]:
%%time

# Step 3: Training and Validation
# Regression (cuRFRegressor)
xgb_reg = cuRFRegressor()
params_reg = {'n_estimators': [5, 10], 'max_depth': [5, 7]}
grid_reg = GridSearchCV(xgb_reg, params_reg, cv=5)
grid_reg.fit(X_reg, y_reg)
best_reg_model = grid_reg.best_estimator_

/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


CPU times: user 1.59 s, sys: 593 ms, total: 2.18 s
Wall time: 1.34 s


## Logistic Regression

In [35]:
X_cls = selected_features[['tip_amount','tolls_amount','total_amount']].to_cupy()
y_cls = selected_features['fare_discretized'].to_cupy()
y_cls = y_cls.get()
X_cls_scaled = X_cls_scaled.get()

AttributeError: DataFrame object has no attribute get

In [36]:
%%time
# Clear previous profiling stats
yappi.clear_stats()
yappi.start()
# Classification (Logistic Regression)
scaler = StandardScaler()
X_cls_scaled = scaler.fit_transform(X_cls)
log_reg = LogisticRegression()
params_cls = {'C': [0.1, 1, 10]}
grid_cls = GridSearchCV(log_reg, params_cls, cv=5)
grid_cls.fit(X_cls_scaled, y_cls)
best_cls_model = grid_cls.best_estimator_
# Stop profiling
yappi.stop()

# Get and sort function stats by total time (descending)
func_stats_lgstc_reg = yappi.get_func_stats().sort('ttot', 'desc')
# Get and sort thread stats by total time (descending)
thread_stats_lgstc_reg = yappi.get_thread_stats().sort('ttot', 'desc')

CPU times: user 6.68 s, sys: 389 ms, total: 7.07 s
Wall time: 7.12 s


## Test and evaluation

In [37]:
%%time
yappi.clear_stats()
yappi.start()
X_cls = selected_features[['tip_amount','tolls_amount','total_amount']].to_cupy()
y_cls = selected_features['fare_discretized'].to_cupy()
# Step 4: Testing
# Example: Split data into train and test sets
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(X_cls_scaled, y_cls, test_size=0.2, random_state=42)


# Evaluate regression model
reg_predictions = best_reg_model.predict(X_test_reg)
reg_rmse = mean_squared_error(y_test_reg, reg_predictions, squared=False)
print(reg_rmse)

# Evaluate classification model
cls_predictions = best_cls_model.predict(X_test_cls)
cls_accuracy = accuracy_score(y_test_cls, cls_predictions)
yappi.stop()
# Get and sort function stats by total time (descending)
func_stats_test_eval = yappi.get_func_stats().sort('ttot', 'desc')
# Get and sort thread stats by total time (descending)
thread_stats_test_eval = yappi.get_thread_stats().sort('ttot', 'desc')


print(cls_accuracy)


4.049892362682655
0.9372000098228455
CPU times: user 1.09 s, sys: 48.4 ms, total: 1.14 s
Wall time: 1.03 s


# Profile Results

## Preprocessing

In [38]:
# Print the top 10 functions
print("Top 10 Function Stats:")
for i, func in enumerate(func_stats_preproc[:10]):
    print(f"Function {i+1}:")
    print(f"  Name: {func.full_name}")
    print(f"  Total Time: {func.ttot}")
    print(f"  Number of Calls: {func.ncall}")
    print(f"  Average Time per Call: {func.tavg}")


# Print the top 10 threads
print("\nTop 10 Thread Stats:")
for i, thread in enumerate(thread_stats_preproc[:10]):
    print(f"Thread {i+1}:")
    print(f"  ID: {thread.id}")
    print(f"  Name: {thread.name}")
    print(f"  Total Time: {thread.ttot}")
    print(f"  Thread ID: {thread.tid}")
    print(f"  Schedule Count: {thread.sched_count}")

Top 10 Function Stats:
Function 1:
  Name: /usr/local/lib/python3.10/dist-packages/nvtx/nvtx.py:113 inner
  Total Time: 0.5538226700000001
  Number of Calls: 1517
  Average Time per Call: 0.0003650775675675676
Function 2:
  Name: /usr/lib/python3.10/concurrent/futures/thread.py:53 _WorkItem.run
  Total Time: 0.24924494200000002
  Number of Calls: 1
  Average Time per Call: 0.24924494200000002
Function 3:
  Name: /usr/local/lib/python3.10/dist-packages/dask/core.py:90 _execute_task
  Total Time: 0.24917211800000003
  Number of Calls: 46
  Average Time per Call: 0.005416785173913044
Function 4:
  Name: /usr/local/lib/python3.10/dist-packages/dask/local.py:235 batch_execute_tasks
  Total Time: 0.24910968700000002
  Number of Calls: 1
  Average Time per Call: 0.24910968700000002
Function 5:
  Name: /usr/local/lib/python3.10/dist-packages/dask/local.py:239 <listcomp>
  Total Time: 0.249106192
  Number of Calls: 1
  Average Time per Call: 0.249106192
Function 6:
  Name: /usr/local/lib/python

## XGBRegressor

In [46]:
# Print the top 10 functions
print("Top 10 Function Stats:")
for i, func in enumerate(func_stats_xgbreg[:10]):
    print(f"Function {i+1}:")
    print(f"  Name: {func.full_name}")
    print(f"  Total Time: {func.ttot}")
    print(f"  Number of Calls: {func.ncall}")
    print(f"  Average Time per Call: {func.tavg}")

# Print the top 10 threads
print("\nTop 10 Thread Stats:")
for i, thread in enumerate(thread_stats_xgbreg[:10]):
    print(f"Thread {i+1}:")
    print(f"  ID: {thread.id}")
    print(f"  Name: {thread.name}")
    print(f"  Total Time: {thread.ttot}")
    print(f"  Thread ID: {thread.tid}")
    print(f"  Schedule Count: {thread.sched_count}")

Top 10 Function Stats:
Function 1:
  Name: /usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:737 GridSearchCV.fit
  Total Time: 0.787902863
  Number of Calls: 1
  Average Time per Call: 0.787902863
Function 2:
  Name: /usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:1386 GridSearchCV._run_search
  Total Time: 0.704942929
  Number of Calls: 1
  Average Time per Call: 0.704942929
Function 3:
  Name: /usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:808 evaluate_candidates
  Total Time: 0.704872982
  Number of Calls: 1
  Average Time per Call: 0.704872982
Function 4:
  Name: /usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:40 Parallel.__call__
  Total Time: 0.700648218
  Number of Calls: 1
  Average Time per Call: 0.700648218
Function 5:
  Name: /usr/local/lib/python3.10/dist-packages/joblib/parallel.py:1902 Parallel.__call__
  Total Time: 0.700616352
  Number of Calls: 1
  Average Time per Call: 

## Logistic Regression

In [40]:
# Print the top 10 functions
print("Top 10 Function Stats:")
for i, func in enumerate(func_stats_lgstc_reg[:10]):
    print(f"Function {i+1}:")
    print(f"  Name: {func.full_name}")
    print(f"  Total Time: {func.ttot}")
    print(f"  Number of Calls: {func.ncall}")
    print(f"  Average Time per Call: {func.tavg}")
# Print the top 10 threads
print("\nTop 10 Thread Stats:")
for i, thread in enumerate(thread_stats_lgstc_reg[:10]):
    print(f"Thread {i+1}:")
    print(f"  ID: {thread.id}")
    print(f"  Name: {thread.name}")
    print(f"  Total Time: {thread.ttot}")
    print(f"  Thread ID: {thread.tid}")
    print(f"  Schedule Count: {thread.sched_count}")

Top 10 Function Stats:
Function 1:
  Name: /usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:737 GridSearchCV.fit
  Total Time: 6.4441451480000005
  Number of Calls: 1
  Average Time per Call: 6.4441451480000005
Function 2:
  Name: /usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:1386 GridSearchCV._run_search
  Total Time: 6.2007920510000005
  Number of Calls: 1
  Average Time per Call: 6.2007920510000005
Function 3:
  Name: /usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:808 evaluate_candidates
  Total Time: 6.200741758
  Number of Calls: 1
  Average Time per Call: 6.200741758
Function 4:
  Name: /usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:40 Parallel.__call__
  Total Time: 6.187079885
  Number of Calls: 1
  Average Time per Call: 6.187079885
Function 5:
  Name: /usr/local/lib/python3.10/dist-packages/joblib/parallel.py:1902 Parallel.__call__
  Total Time: 6.187032530000001
  Number of 

## Test and evaluation

In [41]:
# Print the top 10 functions
print("Top 10 Function Stats:")
for i, func in enumerate(func_stats_test_eval[:10]):
    print(f"Function {i+1}:")
    print(f"  Name: {func.full_name}")
    print(f"  Total Time: {func.ttot}")
    print(f"  Number of Calls: {func.ncall}")
    print(f"  Average Time per Call: {func.tavg}")

# Print the top 10 threads
print("\nTop 10 Thread Stats:")
for i, thread in enumerate(thread_stats_test_eval[:10]):
    print(f"Thread {i+1}:")
    print(f"  ID: {thread.id}")
    print(f"  Name: {thread.name}")
    print(f"  Total Time: {thread.ttot}")
    print(f"  Thread ID: {thread.tid}")
    print(f"  Schedule Count: {thread.sched_count}")

Top 10 Function Stats:
Function 1:
  Name: /usr/local/lib/python3.10/dist-packages/cupy/cuda/compiler.py:471 _compile_module_with_cache
  Total Time: 0.9331032210000001
  Number of Calls: 5
  Average Time per Call: 0.18662064420000002
Function 2:
  Name: /usr/local/lib/python3.10/dist-packages/cupy/cuda/compiler.py:502 _compile_with_cache_cuda
  Total Time: 0.932909352
  Number of Calls: 5
  Average Time per Call: 0.18658187040000002
Function 3:
  Name: /usr/local/lib/python3.10/dist-packages/cupy/cuda/compiler.py:280 compile_using_nvrtc
  Total Time: 0.925709007
  Number of Calls: 5
  Average Time per Call: 0.1851418014
Function 4:
  Name: /usr/local/lib/python3.10/dist-packages/cupy/cuda/compiler.py:283 _compile
  Total Time: 0.922739721
  Number of Calls: 5
  Average Time per Call: 0.1845479442
Function 5:
  Name: /usr/local/lib/python3.10/dist-packages/cupy/cuda/compiler.py:684 _NVRTCProgram.compile
  Total Time: 0.9225511550000001
  Number of Calls: 5
  Average Time per Call: 0.18